In [1]:
from python_files.data import data
data_ = data.copy()
from python_files.matrix.mat_split_data import X_train_mat, X_test_mat, y_train_mat, y_test_mat
from python_files.matrix.mat_col_list import mat_x_col_list, mat_col_list_norm, mat_col_list_std

In [2]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.preprocessing import PowerTransformer, StandardScaler, QuantileTransformer, MaxAbsScaler, RobustScaler
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD, Adam
from scikeras.wrappers import KerasRegressor
from keras.callbacks import ReduceLROnPlateau

### Preprocessing

In [13]:
preprocessor_save_dist = ColumnTransformer(
    transformers=[
        ('normalization', PowerTransformer(), mat_col_list_norm), 
        ('scaling', StandardScaler(), mat_col_list_std) 
    ])

In [5]:
preprocessor_rob = ColumnTransformer(
    transformers=[
        ('scaling', RobustScaler(), mat_x_col_list)
    ])

In [17]:
preprocessor_maxabs = Pipeline(steps = [('scaling_1', StandardScaler(with_mean=True, with_std=False)),
                                     ('scaling_2', MaxAbsScaler())
                                    ])

In [26]:
preprocessor_std = ColumnTransformer(
    transformers=[
        ('scaling', StandardScaler(), mat_x_col_list)
    ])

### Model - MLP

In [6]:
def MLP_model():
    MLP_model = Sequential()
    
    MLP_model.add(Dense(56, input_shape = (12,), activation='selu', bias_initializer='glorot_normal', kernel_initializer='glorot_normal',
                       activity_regularizer='l1_l2'))
    
    MLP_model.add(Dropout(0.4))
   
    MLP_model.add(Dense(44, activation = 'relu', bias_initializer='he_uniform', kernel_initializer='he_uniform',
                       activity_regularizer='l1_l2')) 
    
    #MLP_model.add(Dropout(0.0))
    
    MLP_model.add(Dense(28, activation = 'sigmoid', bias_initializer='glorot_normal', kernel_initializer='he_normal',
                       activity_regularizer='l1_l2')) 
    
    MLP_model.add(Dropout(0.4))
    
    MLP_model.add(Dense(1, activation = 'linear', bias_initializer='he_uniform', kernel_initializer='he_normal',
                       activity_regularizer='l1_l2'))
    
    #adam = Adam(learning_rate = 0.01)
    #sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
    MLP_model.compile(loss = 'mean_absolute_error', optimizer = 'RMSprop', metrics=['mean_absolute_error'])
    return MLP_model

In [7]:
reduce_lr = ReduceLROnPlateau(monitor = 'val_mean_absolute_error', factor = 0.2,
                              patience = 5, min_lr = 0.0000001, min_delta = 0.001)

In [8]:
model = TransformedTargetRegressor(regressor = KerasRegressor(model = MLP_model(), epochs=300, verbose=True),
                                  transformer = QuantileTransformer(n_quantiles = 716, output_distribution = "normal", random_state = 7))

In [27]:
Filling_Matrix_Regressor = Pipeline(steps=[('preprocessor', preprocessor_std),
                           ('model', model)],
                           verbose=True)

### Training

In [28]:
Filling_Matrix_Regressor.fit(X_train_mat, y_train_mat, model__callbacks = [reduce_lr])     #, model__epochs = 200, model__verbose = True) 

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.0s


INFO:tensorflow:Assets written to: C:\Users\WINDOW~1.1\AppData\Local\Temp\tmp3kvetl9_\assets


INFO:tensorflow:Assets written to: C:\Users\WINDOW~1.1\AppData\Local\Temp\tmp3kvetl9_\assets


Epoch 1/300
23/23 [==============================] - 1s 3ms/step - loss: 2.9528 - mean_absolute_error: 1.6419
Epoch 2/300
23/23 [==============================] - 0s 3ms/step - loss: 2.2949 - mean_absolute_error: 1.1321
Epoch 3/300
23/23 [==============================] - 0s 4ms/step - loss: 2.0513 - mean_absolute_error: 1.0005
Epoch 4/300
23/23 [==============================] - 0s 4ms/step - loss: 1.9608 - mean_absolute_error: 1.0155
Epoch 5/300
23/23 [==============================] - 0s 3ms/step - loss: 1.8854 - mean_absolute_error: 1.0252
Epoch 6/300
23/23 [==============================] - 0s 3ms/step - loss: 1.7849 - mean_absolute_error: 0.9970
Epoch 7/300
23/23 [==============================] - 0s 4ms/step - loss: 1.7388 - mean_absolute_error: 1.0049
Epoch 8/300
23/23 [==============================] - 0s 3ms/step - loss: 1.6426 - mean_absolute_error: 0.9628
Epoch 9/300
23/23 [==============================] - 0s 4ms/step - loss: 1.5963 - mean_absolute_error: 0.9649
Epoch 10/3

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('scaling', StandardScaler(),
                                                  Index(['Плотность, кг/м3', 'модуль упругости, ГПа',
       'Количество отвердителя, м.%', 'Содержание эпоксидных групп,%_2',
       'Температура вспышки, С_2', 'Поверхностная плотность, г/м2',
       'Модуль упругости при растяжении, ГПа', 'Прочность при растяжении, МПа',
       'Потребление смолы, г/м2', 'Угол нашивки, град', 'Шаг нашивки',
       'Плотность нашивки'],
      dtype='object'))])),
                ('model',
                 TransformedTargetRegressor(regressor=KerasRegressor(epochs=300, model=<keras.engine.sequential.Sequential object at 0x0000003DBC820370>, verbose=True),
                                            transformer=QuantileTransformer(n_quantiles=716,
                                                                            output_distribution='normal',
                                                                            random_state=7)))],
         verbose=True)

### Prediction

In [29]:
preds = Filling_Matrix_Regressor.predict(X_test_mat)

10/10 [==============================] - 0s 2ms/step


In [30]:
preds_train = Filling_Matrix_Regressor.predict(X_train_mat)

23/23 [==============================] - 0s 2ms/step


### Evaluation

In [31]:
r2_score(y_train_mat, preds_train), r2_score(y_test_mat, preds)

(0.09264480870896352, -0.0650039084696925)

In [32]:
mean_absolute_error(y_train_mat, preds_train), mean_absolute_error(y_test_mat, preds)

(0.7049152082005635, 0.6989356896370008)

In [33]:
np.sqrt(mean_squared_error(y_train_mat, preds_train)), np.sqrt(mean_squared_error(y_test_mat, preds))

(0.8870789911821112, 0.8958564661261903)

In [34]:
mean_absolute_error(y_test_mat, preds) * 100 / (y_test_mat.max() - y_test_mat.min())

13.435027792454662

In [35]:
y_test_mat.describe()

count    307.000000
mean       2.937117
std        0.869503
min        0.389403
25%        2.370830
50%        2.920376
75%        3.495788
max        5.591742
Name: Соотношение матрица-наполнитель, dtype: float64